In [1]:
import pandas as pd
import numpy as np

In [7]:
veriler = pd.read_csv('train.csv', parse_dates=['Tarih'])

In [8]:
veriler = veriler.rename(columns={"Dağıtılan Enerji (MWh)" : "DE"})

In [9]:
def add_quality(o_clock):
    if 0 <= o_clock and o_clock <= 4: 
        return 0
    elif o_clock > 20:
        return 0
    elif 11 < o_clock and o_clock <= 12:
        return 0
    elif 5 < o_clock and o_clock <= 11:
        return 1
    elif 12 < o_clock and o_clock <= 14:
        return 1
    elif 4 < o_clock and o_clock <= 6:
        return 2
    elif 14 < o_clock and o_clock <= 20:
        return 2

In [10]:
veriler['Gün'] = veriler['Tarih'].dt.dayofweek
veriler["GünSırası"]=veriler["Tarih"].dt.day
veriler['Ay'] = veriler['Tarih'].dt.month
#veriler['Yıl'] = veriler['Tarih'].dt.year
veriler['Saat'] = veriler['Tarih'].dt.hour
veriler['Mevsim'] = (veriler['Tarih'].dt.month % 12 + 3) // 3
veriler['Hafta Sonu'] = veriler['Gün'].apply(lambda x: 1 if x in [5, 6] else 0)
veriler['Gece'] = veriler['Saat'].apply(lambda x: 1 if x in [18,19,20,21,22,23,0,1,2,3,4,5] else 0)
veriler["Artmış Mı"] = veriler["Saat"].apply(add_quality)
veriler['Mesai'] = veriler['Saat'].apply(lambda x: 1 if x in [8, 9,10,11,12,14,15,16,17,18,19] else 0)
veriler['Mesai'] = veriler['Mesai'].mask(veriler['Gün'] == 'Pazar', 0)
veriler['Önemli'] = veriler['Ay'].apply(lambda x: "1" if x in [7, 8] else "0")

In [11]:
veriler.set_index('Tarih', inplace=True)

In [12]:
#veriler.info()
veriler

,DE,Gün,GünSırası,Ay,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı,Mesai,Önemli
Tarih,,,,,,,,,,,
2018-01-01 00:00:00,1593.944216,0,1,1,0,1,0,1,0,0,0
2018-01-01 01:00:00,1513.933887,0,1,1,1,1,0,1,0,0,0
2018-01-01 02:00:00,1402.612637,0,1,1,2,1,0,1,0,0,0
2018-01-01 03:00:00,1278.527266,0,1,1,3,1,0,1,0,0,0
2018-01-01 04:00:00,1220.697701,0,1,1,4,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-07-31 19:00:00,2492.489498,6,31,7,19,3,1,1,2,1,1
2022-07-31 20:00:00,2479.958245,6,31,7,20,3,1,1,2,0,1
2022-07-31 21:00:00,2485.016884,6,31,7,21,3,1,1,0,0,1


In [13]:
from sklearn.model_selection import train_test_split
def mape(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [14]:
from fbprophet import Prophet

C:\Users\acer\anaconda3\envs\prophet\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [15]:
test = veriler.iloc[-744:]
train = veriler.iloc[:-744]

In [16]:
model = Prophet()
train = train.reset_index().rename(columns={'Tarih': 'ds', 'DE': 'y','Gün': 'Gün','GünSırası':'GünSırası','Ay': 'Ay','Saat': 'Saat', 'Mevsim':'Mevsim','Hafta Sonu': 'Hafta Sonu', 'Gece': 'Gece', 'Artmış Mı': 'Artmış Mı','Mesai':'Mesai', 'Önemli':'Önemli'})
train

,ds,y,Gün,GünSırası,Ay,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı,Mesai,Önemli
0,2018-01-01 00:00:00,1593.944216,0,1,1,0,1,0,1,0,0,0
1,2018-01-01 01:00:00,1513.933887,0,1,1,1,1,0,1,0,0,0
2,2018-01-01 02:00:00,1402.612637,0,1,1,2,1,0,1,0,0,0
3,2018-01-01 03:00:00,1278.527266,0,1,1,3,1,0,1,0,0,0
4,2018-01-01 04:00:00,1220.697701,0,1,1,4,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
39403,2022-06-30 19:00:00,2392.143374,3,30,6,19,3,0,1,2,1,0
39404,2022-06-30 20:00:00,2349.225766,3,30,6,20,3,0,1,2,0,0
39405,2022-06-30 21:00:00,2365.466215,3,30,6,21,3,0,1,0,0,0
39406,2022-06-30 22:00:00,2229.158338,3,30,6,22,3,0,1,0,0,0


In [18]:
test = test.reset_index().rename(columns={'Tarih': 'ds', 'DE': 'y','Gün': 'Gün','GünSırası':'GünSırası','Ay': 'Ay','Saat': 'Saat', 'Mevsim':'Mevsim','Hafta Sonu': 'Hafta Sonu', 'Gece': 'Gece', 'Artmış Mı': 'Artmış Mı','Mesai':'Mesai', 'Önemli':'Önemli'})
test

,index,ds,y,Gün,GünSırası,Ay,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı,Mesai,Önemli
0,0,2022-07-01 00:00:00,1936.236898,4,1,7,0,3,0,1,0,0,1
1,1,2022-07-01 01:00:00,1786.590420,4,1,7,1,3,0,1,0,0,1
2,2,2022-07-01 02:00:00,1687.829090,4,1,7,2,3,0,1,0,0,1
3,3,2022-07-01 03:00:00,1600.208464,4,1,7,3,3,0,1,0,0,1
4,4,2022-07-01 04:00:00,1584.397059,4,1,7,4,3,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,739,2022-07-31 19:00:00,2492.489498,6,31,7,19,3,1,1,2,1,1
740,740,2022-07-31 20:00:00,2479.958245,6,31,7,20,3,1,1,2,0,1
741,741,2022-07-31 21:00:00,2485.016884,6,31,7,21,3,1,1,0,0,1
742,742,2022-07-31 22:00:00,2396.574255,6,31,7,22,3,1,1,0,0,1


In [19]:
model.fit(train)

C:\Users\acer\anaconda3\envs\prophet\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [20]:
future = model.make_future_dataframe(periods=744, freq='H')
forecast = model.predict(future)
y_pred = forecast['yhat'][-744:]

C:\Users\acer\anaconda3\envs\prophet\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\acer\anaconda3\envs\prophet\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [89]:
y_pred

39408    1978.745368
39409    1852.399738
39410    1766.324133
39411    1705.563132
39412    1659.488846
            ...     
40147    2312.421251
40148    2320.747653
40149    2289.778536
40150    2198.233030
40151    2059.046128
Name: yhat, Length: 744, dtype: float64

In [29]:
Test = test['y'].values

In [30]:
mape(Test, y_pred)

9.965064589785152